In [6]:
# @title
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,780 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,393 kB]
Hi

In [9]:
# Import findspark and initialize.
import findspark
findspark.init()

In [18]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()



In [19]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Load the CSV file into a DataFrame
df = spark.read.csv("./home_sales_revised.csv", header=True, inferSchema=True)


In [20]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")



In [22]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT
    year(date) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year(date)
ORDER BY year
"""

# Execute the query and store the result in a DataFrame
result_df = spark.sql(query)

# Show the result
result_df.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [23]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and store the result in a DataFrame
result_df = spark.sql(query)

# Show the result
result_df.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [24]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and store the result in a DataFrame
result_df = spark.sql(query)

# Show the result
result_df.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [25]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# Query to get the average price of homes per "view" rating
query = """
SELECT
    view AS view_rating,
    ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query and store the result in a DataFrame
result_df = spark.sql(query)

# Show the result
result_df.show()

# End timing
end_time = time.time()

# Print the runtime
print("--- %s seconds ---" % (end_time - start_time))

+-----------+-------------+
|view_rating|average_price|
+-----------+-------------+
|        100|    1026669.5|
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 1.950711727142334 seconds ---


In [26]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")


Is 'home_sales' cached? True


In [27]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [28]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Run the query to get the average price per view rating using the cached data
query_cached = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC
"""

# Execute the query and store the result in a DataFrame
result_df_cached = spark.sql(query_cached)

# Show the result
result_df_cached.show()

# End timing
end_time = time.time()

# Print the runtime for the cached table
print("--- %s seconds ---" % (end_time - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 2.3935022354125977 seconds ---


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Define the path where the partitioned Parquet data will be saved
parquet_path = '/content/home_sales_parquet'

# Write the DataFrame out to Parquet format, partitioned by 'date_built'
df.write.partitionBy('date_built').parquet(parquet_path, mode='overwrite')


In [30]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet(parquet_path)


In [31]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")


In [32]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Define the query
query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales_parquet
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query and store the result in a DataFrame
result_df = spark.sql(query)

# Show the result
result_df.show()

# Print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.1435256004333496 seconds ---


In [33]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [34]:
# 15. Check if the home_sales is no longer cached
cached_tables = spark.catalog.listTables()
home_sales_cached = any(table.name == "home_sales" for table in cached_tables)

if home_sales_cached:
    print("The table 'home_sales' is still cached.")
else:
    print("The table 'home_sales' is no longer cached.")


The table 'home_sales' is still cached.
